# Lab 1: Set Up the Lab Environment

This introductory lab guides the attendees with the following activities:

1. Setting up the AI Project in the AI Foundry
2. Deploying an LLM and embedding models
3. Establish connectivity from VS Code to the AI Project
4. Perform a simple Chat completion call to the model

#### Step 5: Test your lab environment

To test that your lab was setup successfully, run the below code that sends a message to the deployed model, asking it to tell a joke.

In [1]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

True

In [2]:
# Get the project connection string and model from environment variables, which are needed to make a call to the LLM
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
model = os.getenv("CHAT_MODEL")
api_key = os.getenv("CHAT_MODEL_API_KEY")

# Verify variables
if not project_connection_string or not model:
    raise ValueError("Missing required environment variables. Check AIPROJECT_CONNECTION_STRING and CHAT_MODEL in .env file.")

In [ ]:
# Use the connection string to connect to your Foundry project
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise


In [4]:
## need to login using the command azd auth login
agent = project.agents.create_agent(
        model="gpt-4o",
        name="Agent123",
        instructions="You are helpful AI assistant. Answer the user's questions.",
     
    )
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_lKz9CKOEZIEBm2v2bWOotyyf


In [5]:
    # Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_VecAiDfqZ6RaVmn6l0R7llTO


In [6]:
    # Get messages from the thread
message = project.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hey, can you tell a joke about teddy bear?",
    )
print(f"Created message, ID: {message.id}")


Created message, ID: msg_YVLFDY1ufu2GCFBTEp1vMP5g


In [7]:
run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    # [END create_and_process_run]
print(f"Run finished with status: {run.status}")

if run.status == "failed":
        print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [8]:
messages = project.agents.list_messages(thread_id=thread.id)

for msg in messages.data:
    print(f"Message ID: {msg.id}")
    print(f"Role: {msg.role}")
    print("Content:")
    for content in msg.content:
        if content['type'] == 'text':
            print(content['text']['value'])
    print("-" * 50)

# Delete the agent once done
project.agents.delete_agent(agent.id)
("Deleted agent")

Message ID: msg_xnLj6mvbSELrXuCD8t91Oxl9
Role: MessageRole.AGENT
Content:
Sure! Why did the teddy bear say no to dessert?  

Because it was already stuffed!
--------------------------------------------------
Message ID: msg_YVLFDY1ufu2GCFBTEp1vMP5g
Role: MessageRole.USER
Content:
Hey, can you tell a joke about teddy bear?
--------------------------------------------------


'Deleted agent'

### Confirm project connectivity to Azure OpenAI model

In [11]:
aoai_client = project.inference.get_azure_openai_client(api_version="2024-06-01")

response = aoai_client.chat.completions.create(
    model="gpt-4o", # Model deployment name
    messages=[
        {
            "role": "user",
            "content": "How many feet are in a mile?",
        },
    ],
)

print(response.choices[0].message.content)

There are 5,280 feet in a mile.
